# Auto Enocder

In this project, created a specific recommendation system using Auto Encoder algorithm. The data set used in this project was downloaded from the movie lens site. You can download the dataset from [here](https://grouplens.org/datasets/movielens/).

The goal here was to mimic the recommendation systesm of Netflix. Our dataset has similar features as the Netflix dataset: plent of movies. thousands of users, who have rated the movie..

In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

## Part 1 - Data Preprocessing

In [2]:
# Importing the dataset
movies = pd.read_csv('ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')

### Preparing the training set and the test set

In [3]:
training_set = pd.read_csv('ml-100k/u1.base', delimiter = '\t')
training_set = np.array(training_set, dtype = 'int')
test_set = pd.read_csv('ml-100k/u1.test', delimiter = '\t')
test_set = np.array(test_set, dtype = 'int')

### Getting the number of users and movies

In [4]:
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

### Converting the data into an array with users in lines and movies in columns and then converting into Tensors

In [5]:
def convert(data):
    new_data = []
    for id_users in range(1, nb_users + 1):
        id_movies = data[:,1][data[:,0] == id_users]
        id_ratings = data[:,2][data[:,0] == id_users]
        ratings = np.zeros(nb_movies)
        ratings[id_movies - 1] = id_ratings
        new_data.append(list(ratings))
    return new_data
training_set = convert(training_set)
test_set = convert(test_set)

# Converting the data into Torch tensors
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

## Part 2 - Creating the architecture of the Neural Network for Stacked Auto Encoder (SAE)

In [6]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(nb_movies, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, nb_movies)
        self.activation = nn.Sigmoid()
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x

# Initializating the object of class RBM 
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr = 0.01, weight_decay = 0.5)

## Part 3 - Training the SAE

In [8]:
nb_epoch = 200
for epoch in range(1, nb_epoch + 1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set[id_user]).unsqueeze(0)
        target = input.clone()
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output, target)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.data.item()*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))

epoch: 1 loss: 1.7713175386248838
epoch: 2 loss: 1.096561681234173
epoch: 3 loss: 1.0533867734610278
epoch: 4 loss: 1.0380460937833649
epoch: 5 loss: 1.0310473466507193
epoch: 6 loss: 1.0267226648589756
epoch: 7 loss: 1.024018816059181
epoch: 8 loss: 1.0218235481746736
epoch: 9 loss: 1.0207331888294189
epoch: 10 loss: 1.0195961264986304
epoch: 11 loss: 1.0189201286048992
epoch: 12 loss: 1.018366990958366
epoch: 13 loss: 1.0179696858090046
epoch: 14 loss: 1.017450390946745
epoch: 15 loss: 1.0173391100782363
epoch: 16 loss: 1.0170193163468288
epoch: 17 loss: 1.0166796937309373
epoch: 18 loss: 1.0166311765256886
epoch: 19 loss: 1.0163422506273325
epoch: 20 loss: 1.0162177765138345
epoch: 21 loss: 1.016181768824086
epoch: 22 loss: 1.01609408130071
epoch: 23 loss: 1.0160121580961723
epoch: 24 loss: 1.0159246584997639
epoch: 25 loss: 1.0157222241165103
epoch: 26 loss: 1.015586771293697
epoch: 27 loss: 1.015439986695596
epoch: 28 loss: 1.0150046921043896
epoch: 29 loss: 1.0139313393661962
epo

## Part 4 - Testing the SAE

In [13]:
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set[id_user]).unsqueeze(0)
    target = Variable(test_set[id_user]).unsqueeze(0)
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output, target)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.data.item()*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

test loss: 0.949562476453068


Here, test loss is less than 1.*, which means if the user watch recommended movie which suggested by or recommendation, it is likely that the user will rate the recomended movie between 3 and 5 star.